In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import operator

0it [00:00, ?it/s]


In [2]:
with open("train.csv", "r") as f:
    train_text = f.read()
    
rows = train_text.split("\n")
rows_list = []

for i in rows:
    
    data = i.split("|")
    
    if len(data) == 3:
        user_id = data[0]
        movie_name = data[1]
        watch_time = float(data[2])
    
        rows_list.append([user_id, movie_name, watch_time])
        
train = pd.DataFrame(rows_list)
train.columns = ["user_id", "title", "watch_time"]

In [3]:
test = pd.read_csv("test.csv")

In [4]:
with open("movies_meta.csv", "r") as f:
    movies_meta = f.read()

In [5]:
movies = movies_meta.split("\n")[1:]

In [6]:
movies_df = pd.DataFrame()

movies_list = []

for movie in movies:
    
    data = movie.split("|")

    if len(data) == 4:

        movie_name = data[0]
        genre = data[1]

        langs = data[2].split(",")

        movie_length = float(data[3])

        for lang in langs:
            movies_list.append([movie_name, genre, lang, movie_length])

movies_df = pd.DataFrame(movies_list)
movies_df.columns = ["title", "movie_genre", "movie_language", "movie_length"]    

In [7]:
train_with_length = pd.merge(train, movies_df[["title", "movie_length"]], on="title")

In [8]:
train_with_length["ratio_watched"] = train_with_length["watch_time"]/train_with_length["movie_length"]

In [9]:
train_with_length = train_with_length.drop_duplicates()

In [10]:
train_matrix = train_with_length[["user_id", "title", "ratio_watched"]].pivot(index='user_id', columns='title')['ratio_watched']

In [11]:
def num_movies_watched(row):
    return len(row) - row.isnull().sum()

In [12]:
num_movies_watched_df = train_matrix.progress_apply(num_movies_watched, axis = 1)

876597it [04:20, 3371.15it/s]


In [13]:
train_matrix["num_movies"] = num_movies_watched_df

In [23]:
sparse_train_matrix = train_matrix[train_matrix.num_movies > 20]
sparse_train_matrix.drop("num_movies", axis = 1, inplace=True)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
train_matrix.drop("num_movies", axis = 1, inplace=True)

In [12]:
train_matrix.to_pickle("train_matrix.pkl")

SystemError: error return without exception set

In [16]:
def get_cosine(i, j):
    
#     sub_df = pd.concat([sparse_train_matrix.iloc[:, i], sparse_train_matrix.iloc[:, j]], axis = 1)

    return len(sparse_train_matrix[[i, j]].dropna(axis=0, how='any'))

#     sub_df = sub_df.dropna(axis = 0, how = "all")
#     sub_df = sub_df.fillna(0)
#     return 1 - cosine(sub_df.iloc[:, 0], sub_df.iloc[:, 1])

In [ ]:
item_item_sim = {}

col_names = sparse_train_matrix.columns.tolist()

res = {}

def get_similar_items(row):
    
    similar_movies = {}
    
    watched_movies = pd.DataFrame(train_matrix.iloc[train_matrix.index.tolist().index(str(row.user_id)), :].dropna()).index.tolist()
    print row.user_id, watched_movies
    
    for watched_movie in watched_movies:
        
        if watched_movie not in item_item_sim:
            
            item_item_sim[watched_movie] = {}
            
            for i in range(0, len(col_names)):                
                
                item_item_sim[watched_movie][col_names[i]] = get_cosine(watched_movie, col_names[i])
                    
        for j in item_item_sim[watched_movie]:
            
            if j not in similar_movies:
                similar_movies[j] = item_item_sim[watched_movie][j]
            else:
                similar_movies[j] += item_item_sim[watched_movie][j]
    
    
    for watched_movie in watched_movies:
        del similar_movies[watched_movie]
        
    top_similar_movies = sorted(similar_movies.items(), key=operator.itemgetter(1), reverse=True)[:20]
    
    res[str(row.user_id)] = [i[0] for i in top_similar_movies]
    
test.progress_apply(get_similar_items, axis = 1)





  0%|          | 0/6285 [00:00<?, ?it/s]

100146 ['ms dhoni: the untold story', 'nishabd']






  0%|          | 2/6285 [00:07<6:32:39,  3.75s/it]

100166 ['ms dhoni: the untold story']
100245 ['bedroom partner', 'julie']






  0%|          | 4/6285 [00:15<6:38:01,  3.80s/it]

...

140319 ['neerja']
140395 ['10 endrathukulla', 'aamayum muyalum', 'action jackson', 'adi kapyare kootamani', 'akira', 'bang bang', 'bhavani ips', 'bullett raja', 'dragon blade', 'hate story 3', 'holiday', 'housefull 2', 'jai ho', 'jamna pyari', 'janatha garage', 'kaththi', 'kili poyi', 'kunjiramayanam', 'maari', 'oozham', 'shabri', 'singham returns', 'tere bin laden 2', 'the four', 'the king & the commissioner']






  5%|▌         | 325/6285 [30:16<8:10:27,  4.94s/it]

140600 ['baahubali - the beginning', 'magadheera']
140608 ['adam', 'baahubali - the beginning', 'dear dad', 'gang leaders', 'kabali', 'khaidi no. 150', 'tamasha', 'the good dinosaur', 'the jungle book', 'titanic']






  5%|▌         | 327/6285 [30:31<9:27:10,  5.71s/it]

140632 ['yaariyan']
140662 ['anukokunda oka roju', 'malli malli idi rani roju', 'ms dhoni: the untold story', 'singham - yamudu 2']


In [86]:
cosine_vals = []
col_names = sparse_train_matrix.columns.tolist()

for i in range(0, len(train_matrix.columns.tolist())):
    if i%100 == 0:
        print i
    val = get_cosine(col_names[0], col_names[i])
#     val = get_cosine(0, i)

    cosine_vals.append(val)

0
100
200
300
400
500


KeyboardInterrupt: 

In [26]:
res

{'100146': ['jolly llb',
  'hate story 3',
  'baby',
  'dishoom',
  'housefull 3',
  'jolly llb 2',
  'kabali',
  'holiday',
  'prem ratan dhan payo',
  'akira',
  'bang bang',
  'neerja',
  '2 states',
  'kick',
  'drishyam',
  'sanam re',
  'raaz: the mystery continues',
  'dolly ki doli',
  'bedroom partner',
  'highway'],
 '100166': ['jolly llb',
  'dishoom',
  'hate story 3',
  'baby',
  'housefull 3',
  'jolly llb 2',
  'kabali',
  'prem ratan dhan payo',
  'bang bang',
  'holiday',
  'kick',
  '2 states',
  'neerja',
  'akira',
  'drishyam',
  'sanam re',
  'bajrangi bhaijaan',
  'dolly ki doli',
  'singham returns',
  'raaz: the mystery continues'],
 '100245': ['hate story 3',
  'jolly llb',
  'ms dhoni: the untold story',
  'raaz: the mystery continues',
  'housefull 3',
  'baby',
  'dishoom',
  'bang bang',
  'sanam re',
  'kabali',
  'prem ratan dhan payo',
  'jolly llb 2',
  'akira',
  'holiday',
  'kick',
  'neerja',
  'dolly ki doli',
  'hero',
  '2 states',
  'jaggu dada